In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from deepctr_torch.inputs import SparseFeat, get_feature_names
from deepctr_torch.models import DeepFM
import torch

# Metricas

In [ ]:
animes = pd.read_csv('clean_data/animes.csv')

In [ ]:
df_train = pd.read_csv(
    "train", sep=",", names=["userid", "itemid", "rating"], header=None
)

# Convert ratings to binary target (1 if >= 5 else 0)
df_train.rating = [1 if x >= 5 else 0 for x in df_train.rating]

df_train.head()

,userid,itemid,rating
0,14179,13601.0,1
1,37548,34300.0,1
2,796,2592.0,0
3,3041,949.0,1
4,2493,5114.0,1


In [ ]:
df_test = pd.read_csv(
    "test", sep=",", names=["userid", "itemid", "rating"], header=None
)

df_test.head()

,userid,itemid,rating
0,506,37517.0,10
1,16392,7311.0,9
2,553,12471.0,5
3,13348,8937.0,6
4,276,35997.0,3


In [ ]:
item_interaction_counts = df_train['itemid'].value_counts()
user_count = df_train['userid'].nunique()
item_popularity = (item_interaction_counts / user_count).to_dict()
metadata = animes[['uid', 'genre']]
item_categories: dict[int, set[str | None]] = {}
for row in metadata.itertuples():
    item_categories[int(row[1]) if hasattr(row[1], 'is_integer') and row[1].is_integer() else row[1]] = set(map(lambda i: i.strip(), row[2].split(','))) if isinstance(row[2], str) else set()

In [ ]:
df_train = df_train.dropna(subset=['itemid'])

In [ ]:
data = pd.concat([df_train, df_test], axis=0, ignore_index=True)

lbe_user = LabelEncoder()
data['userid_enc'] = lbe_user.fit_transform(data['userid'])

lbe_item = LabelEncoder()
data['itemid_enc'] = lbe_item.fit_transform(data['itemid'])

train = data.iloc[:len(df_train)].copy()
test = data.iloc[len(df_train):].copy()

feature_columns = [
    SparseFeat("userid_enc", vocabulary_size=data['userid_enc'].max() + 1, embedding_dim=32),
    SparseFeat("itemid_enc", vocabulary_size=data['itemid_enc'].max() + 1, embedding_dim=32)
]

linear_feature_columns = feature_columns
dnn_feature_columns = feature_columns
feature_names = get_feature_names(linear_feature_columns)

train_model_input = {name: train[name].values for name in feature_names}
test_model_input = {name: test[name].values for name in feature_names}

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary', device=device)

model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'])

# Training
history = model.fit(train_model_input, train['rating'].values, batch_size=256, epochs=15, verbose=1)

cpu
Train on 139689 samples, validate on 0 samples, 546 steps per epoch


546it [00:02, 226.86it/s]


Epoch 1/3
2s - loss:  0.2923 - binary_crossentropy:  0.2922


546it [00:02, 228.21it/s]


Epoch 2/3
2s - loss:  0.1906 - binary_crossentropy:  0.1906


546it [00:02, 205.41it/s]

Epoch 3/3
2s - loss:  0.1587 - binary_crossentropy:  0.1587


In [ ]:
user_items_test = {}
for row in df_test.itertuples():
    if row.userid not in user_items_test:
        user_items_test[row.userid] = []
    user_items_test[row.userid].append(row.itemid)

all_items_enc = data['itemid_enc'].unique()
enc_to_raw_item = {enc: raw for enc, raw in zip(data['itemid_enc'], data['itemid'])}

In [ ]:
def get_recommendations(user_id, n):
    try:
        user_enc = lbe_user.transform([user_id])[0]
    except ValueError:
        return np.array([])

    # Create input for ALL items for this user
    user_enc_col = np.full(len(all_items_enc), user_enc)
    
    pred_input = {
        "userid_enc": user_enc_col,
        "itemid_enc": all_items_enc
    }
    
    # Predict
    preds = model.predict(pred_input, batch_size=4096).flatten()
    
    # Rank
    top_indices = preds.argsort()[-n:][::-1]
    top_enc_items = all_items_enc[top_indices]
    
    recommendations = [enc_to_raw_item.get(i) for i in top_enc_items]
    return np.array(recommendations)

In [ ]:
from evaluate import get_metrics

get_metrics(user_items_test, item_popularity, item_categories, get_recommendations, k=10, delta=0.05)

--- Métricas Globales de Evaluación ---
{
  "mean_recall": 0.018221878163520513,
  "mean_precision": 0.0018519621270767584,
  "mean_ap (MAP)": 0.006700684998189911,
  "mean_ndcg": 0.009354649043280561,
  "mean_novelty": 7.565578571071954,
  "mean_diversity": 0.9003142859663432,
  "num_users_evaluated": 16793
}

--- Reporte de Fairness (Disparidad de Grupo) ---
{
  "delta_threshold": 0.05,
  "is_biased_recall": 0,
  "is_biased_precision": 0,
  "group_averages": {
    "Male": {
      "recall (Cobertura)": 0.019952906184161606,
      "precision (Tasa Aceptaci\u00f3n)": 0.0020572561655719424,
      "count": 8069
    },
    "NaN": {
      "recall (Cobertura)": 0.016129032258064516,
      "precision (Tasa Aceptaci\u00f3n)": 0.0016129032258064516,
      "count": 5518
    },
    "Non-Binary": {
      "recall (Cobertura)": 0.021897810218978103,
      "precision (Tasa Aceptaci\u00f3n)": 0.0021897810218978104,
      "count": 137
    },
    "Female": {
      "recall (Cobertura)": 0.017269468882372